# 09 — Methods Comparison (All 13 Estimators)

This notebook compares **all 13 fat-tail estimators** in the fatcrash library across **4 assets**.

**Estimators covered:**

| # | Method | Category |
|---|--------|----------|
| 1 | Hill estimator | Tail index |
| 2 | DEH (Dekkers-Einmahl-de Haan) | Tail index |
| 3 | QQ estimator | Tail index |
| 4 | Pickands estimator | Tail index |
| 5 | Max-stability kappa | Kappa metric |
| 6 | Taleb kappa | Kappa metric |
| 7 | Max-to-Sum ratio | Concentration |
| 8 | GPD fit | Extreme Value Theory |
| 9 | GPD VaR/ES | Extreme Value Theory |
| 10 | GEV fit | Extreme Value Theory |
| 11 | Hurst exponent | Persistence / memory |
| 12 | DFA exponent | Persistence / memory |
| 13 | Spectral exponent | Persistence / memory |

**Assets:** BTC, SPY, Gold, GBP/USD

## 1. Imports and Data Loading

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.style.use("seaborn-v0_8-whitegrid")

from fatcrash.data.ingest import from_sample, from_csv, load_fred_forex
from fatcrash.data.transforms import log_returns, negative_returns, block_maxima
from fatcrash._core import (
    hill_estimator, hill_rolling,
    kappa_metric, kappa_rolling,
    taleb_kappa, taleb_kappa_rolling,
    pickands_estimator, pickands_rolling,
    hurst_exponent, hurst_rolling,
    dfa_exponent, dfa_rolling,
    deh_estimator, deh_rolling,
    qq_estimator, qq_rolling,
    maxsum_ratio, maxsum_rolling,
    spectral_exponent, spectral_rolling,
    gpd_fit, gpd_var_es, gev_fit,
    lppls_fit,
)

In [ ]:
# Load all 4 assets
btc = from_sample('btc')
spy = from_sample('spy')
gold = from_sample('gold')
gbpusd = from_csv('data/sample/gbpusd_daily.csv')

assets = {'BTC': btc, 'SPY': spy, 'Gold': gold, 'GBP/USD': gbpusd}

# Compute returns for each asset
returns = {}
for name, df in assets.items():
    returns[name] = log_returns(df)

for name, df in assets.items():
    ret = returns[name]
    print(f"{name}: {len(df)} days, {len(ret)} returns, "
          f"{df.index[0].date()} to {df.index[-1].date()}")

## 2. Tail Index Comparison (Hill, DEH, QQ)

Three estimators of the tail index alpha. Lower alpha means fatter tails.
- **alpha < 2**: infinite variance (extremely fat)
- **alpha < 4**: fat tails, finite variance but infinite kurtosis
- **alpha > 4**: approaching thin-tailed behaviour

In [ ]:
tail_results = []
for name in assets:
    ret = returns[name]
    hill_a = hill_estimator(ret)
    deh_a = deh_estimator(ret)
    qq_a = qq_estimator(ret)
    tail_results.append({
        'Asset': name,
        'Hill alpha': round(hill_a, 3),
        'DEH alpha': round(deh_a, 3),
        'QQ alpha': round(qq_a, 3),
        'N': len(ret),
    })

tail_df = pd.DataFrame(tail_results)
print(tail_df.to_string(index=False))

In [ ]:
# Side-by-side bar chart
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(tail_df))
width = 0.25

ax.bar(x - width, tail_df['Hill alpha'], width, label='Hill', color='#2196F3')
ax.bar(x, tail_df['DEH alpha'], width, label='DEH', color='#FF9800')
ax.bar(x + width, tail_df['QQ alpha'], width, label='QQ', color='#4CAF50')

ax.axhline(y=2, color='red', linestyle='--', linewidth=1, label='alpha=2 (infinite variance)')
ax.axhline(y=4, color='green', linestyle='--', linewidth=1, label='alpha=4 (thin tail boundary)')

ax.set_xticks(x)
ax.set_xticklabels(tail_df['Asset'])
ax.set_ylabel('Tail index alpha (lower = fatter tails)')
ax.set_title('Tail Index Comparison: Hill vs DEH vs QQ')
ax.legend()
plt.tight_layout()
plt.show()

## 3. Kappa Metrics (Max-stability Kappa + Taleb Kappa)

- **Max-stability kappa**: kappa below the Gaussian benchmark signals fat tails.
- **Taleb kappa**: kappa above the benchmark signals fat tails (inverse convention).

Both measure departure from Gaussian max-stability, but with opposite sign conventions.

In [ ]:
kappa_results = []
for name in assets:
    ret = returns[name]
    k_val, k_bench = kappa_metric(ret, n_subsamples=10)
    tk_val, tk_bench = taleb_kappa(ret)
    kappa_results.append({
        'Asset': name,
        'Kappa': round(k_val, 4),
        'Kappa Bench': round(k_bench, 4),
        'Kappa < Bench?': 'FAT' if k_val < k_bench else 'thin',
        'Taleb Kappa': round(tk_val, 4),
        'Taleb Bench': round(tk_bench, 4),
        'Taleb > Bench?': 'FAT' if tk_val > tk_bench else 'thin',
    })

kappa_df = pd.DataFrame(kappa_results)
print(kappa_df.to_string(index=False))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

x = np.arange(len(kappa_df))
width = 0.35

# Max-stability kappa
ax1.bar(x - width/2, kappa_df['Kappa'], width, label='Kappa', color='#2196F3')
ax1.bar(x + width/2, kappa_df['Kappa Bench'], width, label='Gaussian Benchmark', color='#9E9E9E')
ax1.set_xticks(x)
ax1.set_xticklabels(kappa_df['Asset'])
ax1.set_ylabel('Kappa')
ax1.set_title('Max-stability Kappa (below bench = fat tails)')
ax1.legend()

# Taleb kappa
ax2.bar(x - width/2, kappa_df['Taleb Kappa'], width, label='Taleb Kappa', color='#FF5722')
ax2.bar(x + width/2, kappa_df['Taleb Bench'], width, label='Gaussian Benchmark', color='#9E9E9E')
ax2.set_xticks(x)
ax2.set_xticklabels(kappa_df['Asset'])
ax2.set_ylabel('Taleb Kappa')
ax2.set_title('Taleb Kappa (above bench = fat tails)')
ax2.legend()

plt.tight_layout()
plt.show()

## 4. Max-to-Sum Ratio

The max-to-sum ratio measures how much a single observation dominates the total.
- For thin-tailed data, the ratio converges to 0 as N grows.
- For fat-tailed data (alpha < 2), the ratio stays bounded away from 0.
- Higher ratios indicate heavier concentration of risk in single events.

In [ ]:
maxsum_results = []
for name in assets:
    ret = returns[name]
    ratio = maxsum_ratio(ret)
    maxsum_results.append({'Asset': name, 'Max-to-Sum Ratio': round(ratio, 5)})

maxsum_df = pd.DataFrame(maxsum_results)
print(maxsum_df.to_string(index=False))

fig, ax = plt.subplots(figsize=(8, 4))
colors = ['#E53935' if r > 0.05 else '#43A047' for r in maxsum_df['Max-to-Sum Ratio']]
ax.bar(maxsum_df['Asset'], maxsum_df['Max-to-Sum Ratio'], color=colors)
ax.axhline(y=0.05, color='orange', linestyle='--', linewidth=1, label='0.05 threshold')
ax.set_ylabel('Max-to-Sum Ratio')
ax.set_title('Max-to-Sum Ratio by Asset')
ax.legend()
plt.tight_layout()
plt.show()

print("\nInterpretation: A higher ratio means a single extreme event accounts for")
print("a larger fraction of the total sum of absolute returns. Values above ~0.05")
print("suggest meaningful tail concentration.")

## 5. EVT: VaR, Expected Shortfall, and GEV Shape

**GPD** (Generalized Pareto Distribution) models exceedances over a threshold to estimate:
- **VaR**: Value-at-Risk at the 99% confidence level
- **ES**: Expected Shortfall (average loss beyond VaR)

**GEV** (Generalized Extreme Value) fits block maxima:
- **xi > 0**: Frechet (fat tails)
- **xi ~ 0**: Gumbel (exponential tails)
- **xi < 0**: Weibull (bounded tails)

In [ ]:
evt_results = []
for name in assets:
    ret = returns[name]
    try:
        sigma, xi, threshold, n_exc = gpd_fit(ret, quantile=0.95)
        var99, es99 = gpd_var_es(ret, p=0.99, quantile=0.95)
        var95, es95 = gpd_var_es(ret, p=0.95, quantile=0.95)
        evt_results.append({
            'Asset': name,
            'GPD sigma': round(sigma, 5),
            'GPD xi': round(xi, 3),
            'VaR 95%': round(var95, 4),
            'VaR 99%': round(var99, 4),
            'ES 99%': round(es99, 4),
            'Exceedances': n_exc,
        })
    except Exception as e:
        evt_results.append({'Asset': name, 'Error': str(e)})

evt_df = pd.DataFrame(evt_results)
print("GPD Results:")
print(evt_df.to_string(index=False))

In [ ]:
# VaR and ES bar chart
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(evt_df))
width = 0.3

ax.bar(x - width, evt_df['VaR 95%'], width, label='VaR 95%', color='#FFC107')
ax.bar(x, evt_df['VaR 99%'], width, label='VaR 99%', color='#FF5722')
ax.bar(x + width, evt_df['ES 99%'], width, label='ES 99%', color='#B71C1C')

ax.set_xticks(x)
ax.set_xticklabels(evt_df['Asset'])
ax.set_ylabel('Loss (log returns)')
ax.set_title('Tail Risk: GPD-based VaR and Expected Shortfall')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# GEV shape parameter
gev_results = []
for name in assets:
    ret = returns[name]
    bm = block_maxima(ret, block_size=21)
    mu, sigma, xi = gev_fit(bm)
    tail_type = 'Frechet (fat)' if xi > 0.05 else ('Gumbel' if xi > -0.05 else 'Weibull (bounded)')
    gev_results.append({
        'Asset': name,
        'GEV mu': round(mu, 5),
        'GEV sigma': round(sigma, 5),
        'GEV xi': round(xi, 3),
        'Tail type': tail_type,
    })

gev_df = pd.DataFrame(gev_results)
print("GEV Block Maxima Results (block_size=21):")
print(gev_df.to_string(index=False))

## 6. Persistence: Hurst, DFA, Spectral

Three measures of long-range dependence / persistence:
- **Hurst exponent**: H = 0.5 (random walk), H > 0.5 (persistent), H < 0.5 (mean-reverting)
- **DFA exponent**: similar interpretation to Hurst, estimated via detrended fluctuation analysis
- **Spectral exponent**: estimated from the power spectrum; higher values indicate more persistence

In [ ]:
persist_results = []
for name in assets:
    ret = returns[name]
    h = hurst_exponent(ret)
    d = dfa_exponent(ret)
    s = spectral_exponent(ret)
    persist_results.append({
        'Asset': name,
        'Hurst H': round(h, 4),
        'DFA alpha': round(d, 4),
        'Spectral beta': round(s, 4),
    })

persist_df = pd.DataFrame(persist_results)
print(persist_df.to_string(index=False))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(persist_df))
width = 0.25

ax.bar(x - width, persist_df['Hurst H'], width, label='Hurst H', color='#673AB7')
ax.bar(x, persist_df['DFA alpha'], width, label='DFA alpha', color='#00BCD4')
ax.bar(x + width, persist_df['Spectral beta'], width, label='Spectral beta', color='#8BC34A')

ax.axhline(y=0.5, color='grey', linestyle='--', linewidth=1, label='H=0.5 (random walk)')

ax.set_xticks(x)
ax.set_xticklabels(persist_df['Asset'])
ax.set_ylabel('Exponent value')
ax.set_title('Persistence Measures: Hurst, DFA, Spectral')
ax.legend(loc='upper right')
plt.tight_layout()
plt.show()

## 7. Rolling Indicators: Hill Alpha Across Assets

252-day rolling window of Hill tail index alpha for all 4 assets.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 8), sharex=False, sharey=True)
axes = axes.flatten()

for i, (name, df) in enumerate(assets.items()):
    ret = returns[name]
    alpha_rolling = np.asarray(hill_rolling(ret, window=252))
    dates = df.index[1:]  # returns are 1 shorter than the DataFrame

    ax = axes[i]
    ax.plot(dates, alpha_rolling, linewidth=0.8, color='#1565C0')
    ax.axhline(y=2, color='red', linestyle='--', linewidth=0.8, label='alpha=2')
    ax.axhline(y=4, color='green', linestyle='--', linewidth=0.8, label='alpha=4')
    ax.set_title(name)
    ax.set_ylabel('Hill alpha')
    ax.set_ylim(0, 8)
    ax.legend(loc='upper right', fontsize=8)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.tick_params(axis='x', rotation=45)

fig.suptitle('Rolling Hill Alpha (252-day window)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 8. 2017 BTC Bubble Deep Dive

Zoom into the 2017 BTC bubble (2017-01-01 to 2018-03-01) and overlay multiple rolling indicators
to see how they behave before, during, and after the crash.

Vertical red line marks the peak at **2017-12-17**.

In [ ]:
btc_2017 = btc.loc['2017-01-01':'2018-03-01']
ret_2017 = log_returns(btc_2017)
dates_2017 = btc_2017.index[1:]  # match returns length

# Rolling indicators (60-day window)
hill_60 = np.asarray(hill_rolling(ret_2017, window=60))
dfa_60 = np.asarray(dfa_rolling(ret_2017, window=60))
kappa_60, kappa_bench_60 = kappa_rolling(ret_2017, window=60, n_subsamples=5)
kappa_60 = np.asarray(kappa_60)

peak_date = pd.Timestamp('2017-12-17')

fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# Subplot 1: Price
axes[0].plot(btc_2017.index, btc_2017['close'], color='#FF9800', linewidth=1.2)
axes[0].set_ylabel('BTC Price (USD)')
axes[0].set_title('BTC Price')
axes[0].axvline(x=peak_date, color='red', linestyle='--', linewidth=1, label='Peak (2017-12-17)')
axes[0].legend(loc='upper left')

# Subplot 2: Rolling Hill alpha
axes[1].plot(dates_2017, hill_60, color='#1565C0', linewidth=1)
axes[1].axhline(y=2, color='red', linestyle='--', linewidth=0.7, alpha=0.7)
axes[1].axhline(y=4, color='green', linestyle='--', linewidth=0.7, alpha=0.7)
axes[1].set_ylabel('Hill alpha')
axes[1].set_title('Rolling Hill Alpha (60d)')
axes[1].axvline(x=peak_date, color='red', linestyle='--', linewidth=1)

# Subplot 3: Rolling DFA
axes[2].plot(dates_2017, dfa_60, color='#00BCD4', linewidth=1)
axes[2].axhline(y=0.5, color='grey', linestyle='--', linewidth=0.7, alpha=0.7, label='H=0.5')
axes[2].set_ylabel('DFA exponent')
axes[2].set_title('Rolling DFA Exponent (60d)')
axes[2].axvline(x=peak_date, color='red', linestyle='--', linewidth=1)
axes[2].legend(loc='upper left')

# Subplot 4: Rolling Kappa
axes[3].plot(dates_2017, kappa_60, color='#E91E63', linewidth=1, label='Kappa')
axes[3].axhline(y=kappa_bench_60, color='grey', linestyle='--', linewidth=0.7, label=f'Gaussian bench ({kappa_bench_60:.3f})')
axes[3].set_ylabel('Kappa')
axes[3].set_title('Rolling Max-stability Kappa (60d)')
axes[3].axvline(x=peak_date, color='red', linestyle='--', linewidth=1)
axes[3].legend(loc='upper left')

axes[3].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
axes[3].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 9. Method Agreement Matrix

For each asset, classify each method as detecting fat tails (FAT) or not (thin).

**Thresholds used:**
- Hill, DEH, QQ: alpha < 4
- Max-stability Kappa: kappa < benchmark
- Taleb Kappa: kappa > benchmark
- Pickands: gamma > 0 (i.e., xi = 1/gamma implies fat tail)
- Max-to-Sum: ratio > 0.05
- GPD: xi > 0
- GEV: xi > 0

In [ ]:
agreement = []
for name in assets:
    ret = returns[name]

    # Tail index estimators
    hill_a = hill_estimator(ret)
    deh_a = deh_estimator(ret)
    qq_a = qq_estimator(ret)
    pick_g = pickands_estimator(ret)

    # Kappa metrics
    k_val, k_bench = kappa_metric(ret, n_subsamples=10)
    tk_val, tk_bench = taleb_kappa(ret)

    # Max-to-Sum
    ms = maxsum_ratio(ret)

    # EVT
    try:
        _, gpd_xi, _, _ = gpd_fit(ret, quantile=0.95)
        gpd_fat = gpd_xi > 0
    except Exception:
        gpd_xi = float('nan')
        gpd_fat = None

    bm = block_maxima(ret, block_size=21)
    _, _, gev_xi = gev_fit(bm)

    row = {
        'Asset': name,
        'Hill (a<4)': 'FAT' if hill_a < 4 else 'thin',
        'DEH (a<4)': 'FAT' if deh_a < 4 else 'thin',
        'QQ (a<4)': 'FAT' if qq_a < 4 else 'thin',
        'Pickands (g>0)': 'FAT' if pick_g > 0 else 'thin',
        'Kappa (<bench)': 'FAT' if k_val < k_bench else 'thin',
        'Taleb (>bench)': 'FAT' if tk_val > tk_bench else 'thin',
        'MaxSum (>0.05)': 'FAT' if ms > 0.05 else 'thin',
        'GPD (xi>0)': 'FAT' if gpd_fat else ('thin' if gpd_fat is not None else 'N/A'),
        'GEV (xi>0)': 'FAT' if gev_xi > 0 else 'thin',
    }

    # Count agreement
    fat_count = sum(1 for k, v in row.items() if k != 'Asset' and v == 'FAT')
    total = sum(1 for k, v in row.items() if k != 'Asset' and v != 'N/A')
    row['Agreement'] = f'{fat_count}/{total}'
    agreement.append(row)

agree_df = pd.DataFrame(agreement)
print(agree_df.to_string(index=False))

## 10. Edge Cases: Known Distributions

Run all estimators on synthetic data with known properties to verify they produce
sensible results:
- **Gaussian**: alpha = infinity (no fat tails)
- **Student-t(3)**: alpha = 3 (fat tails, finite variance, infinite kurtosis)
- **Cauchy**: alpha = 1 (extremely fat tails, infinite variance)
- **Pareto(2)**: alpha = 2 (fat tails, infinite variance)

In [ ]:
rng = np.random.default_rng(42)
n = 5000

distributions = {
    'Gaussian': rng.standard_normal(n),
    'Student-t(3)': rng.standard_t(3, n),
    'Cauchy': rng.standard_cauchy(n),
    'Pareto(2)': rng.pareto(2, n),
}

edge_results = []
for dist_name, data in distributions.items():
    # Tail index estimators
    hill_a = hill_estimator(data)
    deh_a = deh_estimator(data)
    qq_a = qq_estimator(data)
    pick_g = pickands_estimator(data)

    # Kappa metrics
    k_val, k_bench = kappa_metric(data, n_subsamples=10)
    tk_val, tk_bench = taleb_kappa(data)

    # Max-to-Sum
    ms = maxsum_ratio(data)

    # EVT
    try:
        _, gpd_xi, _, _ = gpd_fit(data, quantile=0.95)
    except Exception:
        gpd_xi = float('nan')

    try:
        bm = block_maxima(data, block_size=21)
        _, _, gev_xi = gev_fit(bm)
    except Exception:
        gev_xi = float('nan')

    # Persistence
    h = hurst_exponent(data)
    d = dfa_exponent(data)
    s = spectral_exponent(data)

    edge_results.append({
        'Distribution': dist_name,
        'Hill alpha': round(hill_a, 2),
        'DEH alpha': round(deh_a, 2),
        'QQ alpha': round(qq_a, 2),
        'Pickands gamma': round(pick_g, 3),
        'Kappa': round(k_val, 4),
        'K Bench': round(k_bench, 4),
        'Taleb K': round(tk_val, 4),
        'TK Bench': round(tk_bench, 4),
        'MaxSum': round(ms, 4),
        'GPD xi': round(gpd_xi, 3),
        'GEV xi': round(gev_xi, 3),
        'Hurst': round(h, 3),
        'DFA': round(d, 3),
        'Spectral': round(s, 3),
    })

edge_df = pd.DataFrame(edge_results)
print(edge_df.to_string(index=False))

## 10b. FRED Forex: All 23 Currency Pairs

The four sample assets and synthetic distributions above test our methods on a handful
of series. But to establish that **fat tails are universal in exchange rates**, we need
a broader sample.

We load **23 daily currency pairs** from FRED via the
[forex-centuries](https://github.com/unbalancedparentheses/forex-centuries) repository
and run every estimator on each pair. This is the same dataset used in the
[accuracy report](../analysis/accuracy_report.py).

**Data requirements:**
```bash
git clone https://github.com/unbalancedparentheses/forex-centuries ~/projects/forex-centuries
```
Or set `FOREX_CENTURIES_DIR` to your clone location.

In [ ]:
# Load all 23 FRED forex pairs
fred_pairs = load_fred_forex()
print(f"Loaded {len(fred_pairs)} FRED currency pairs\n")

# Run all estimators (tail, kappa, EVT, persistence) on each pair
fred_rows = []
for pair_name, pair_df in sorted(fred_pairs.items()):
    r = log_returns(pair_df)
    l = negative_returns(r)

    if len(l) < 100:
        print(f"  Skipping {pair_name}: only {len(l)} observations")
        continue

    # Tail index estimators
    hill_a = hill_estimator(l)
    deh_g = deh_estimator(l)
    qq_a = qq_estimator(l)
    pick_g = pickands_estimator(l)

    # Kappa metrics
    k_val, k_bench = kappa_metric(r, n_subsamples=10)
    tk_val, tk_bench = taleb_kappa(r)

    # Max-to-Sum
    ms = maxsum_ratio(l)

    # EVT
    try:
        sigma, gpd_xi, _, n_exc = gpd_fit(r, quantile=0.95)
        var99, es99 = gpd_var_es(r, p=0.99, quantile=0.95)
    except Exception:
        gpd_xi, var99, es99 = float("nan"), float("nan"), float("nan")

    try:
        bm = block_maxima(r, block_size=21)
        _, _, gev_xi = gev_fit(bm)
    except Exception:
        gev_xi = float("nan")

    # Persistence
    h = hurst_exponent(r)
    d = dfa_exponent(r)
    s = spectral_exponent(r)

    fred_rows.append({
        "Pair": pair_name,
        "N": len(r),
        "Hill alpha": hill_a,
        "QQ alpha": qq_a,
        "DEH gamma": deh_g,
        "Pickands gamma": pick_g,
        "Kappa": k_val,
        "Taleb K": tk_val,
        "MaxSum": ms,
        "GPD xi": gpd_xi,
        "GEV xi": gev_xi,
        "VaR 99%": var99,
        "ES 99%": es99,
        "Hurst H": h,
        "DFA alpha": d,
        "Spectral d": s,
    })

fred_all = pd.DataFrame(fred_rows).set_index("Pair")

print(f"FRED Forex: All {len(fred_all)} Pairs — Complete Estimator Comparison")
print("=" * 100)
display(fred_all.style.format(
    {col: "{:.4f}" for col in fred_all.columns if col != "N"}
).format({"N": "{:,d}"}))

In [ ]:
# Method agreement matrix for all 23 FRED forex pairs
fred_agree = []
for pair_name in fred_all.index:
    row = fred_all.loc[pair_name]

    checks = {
        "Hill (a<4)":     row["Hill alpha"] < 4,
        "QQ (a<4)":       row["QQ alpha"] < 4,
        "DEH (g>0)":      row["DEH gamma"] > 0,
        "Pickands (g>0)": row["Pickands gamma"] > 0,
        "GPD (xi>0)":     row["GPD xi"] > 0 if not np.isnan(row["GPD xi"]) else None,
        "GEV (xi>0)":     row["GEV xi"] > 0 if not np.isnan(row["GEV xi"]) else None,
        "Hurst (>0.5)":   row["Hurst H"] > 0.5,
        "DFA (>0.5)":     row["DFA alpha"] > 0.5,
    }

    fat_count = sum(1 for v in checks.values() if v is True)
    total = sum(1 for v in checks.values() if v is not None)

    fred_agree.append({
        "Pair": pair_name,
        **{k: ("FAT" if v else "thin") if v is not None else "N/A" for k, v in checks.items()},
        "Agreement": f"{fat_count}/{total}",
    })

fred_agree_df = pd.DataFrame(fred_agree).set_index("Pair")

print("Method Agreement: FRED Forex Pairs")
print("=" * 100)
print(f"(FAT = method detects fat tails / persistence)\n")
display(fred_agree_df)

# Count how many pairs each method flags
print("\nMethod detection rates:")
for col in fred_agree_df.columns:
    if col == "Agreement":
        continue
    fat_count = (fred_agree_df[col] == "FAT").sum()
    total = (fred_agree_df[col] != "N/A").sum()
    print(f"  {col:20s}: {fat_count}/{total} pairs ({fat_count/total*100:.0f}%)")

In [ ]:
# Visualizations for FRED forex results
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Panel 1: Hill alpha sorted bar chart
sorted_hill = fred_all["Hill alpha"].sort_values()
colors_hill = ["#E53935" if a < 2 else "#FF9800" if a < 4 else "#43A047" for a in sorted_hill]
axes[0, 0].barh(sorted_hill.index, sorted_hill.values, color=colors_hill, edgecolor="black", linewidth=0.3)
axes[0, 0].axvline(2, color="red", linestyle="--", alpha=0.7, label="alpha=2")
axes[0, 0].axvline(4, color="green", linestyle="--", alpha=0.7, label="alpha=4")
axes[0, 0].set_xlabel("Hill alpha")
axes[0, 0].set_title("Hill Tail Index (sorted)")
axes[0, 0].legend(fontsize=8)
axes[0, 0].tick_params(axis="y", labelsize=8)

# Panel 2: Hurst H sorted bar chart
sorted_hurst = fred_all["Hurst H"].sort_values()
colors_hurst = ["#2196F3" if h > 0.5 else "#9E9E9E" for h in sorted_hurst]
axes[0, 1].barh(sorted_hurst.index, sorted_hurst.values, color=colors_hurst, edgecolor="black", linewidth=0.3)
axes[0, 1].axvline(0.5, color="red", linestyle="--", alpha=0.7, label="H=0.5")
axes[0, 1].set_xlabel("Hurst H")
axes[0, 1].set_title("Hurst Exponent (sorted)")
axes[0, 1].legend(fontsize=8)
axes[0, 1].tick_params(axis="y", labelsize=8)

# Panel 3: VaR 99% and ES 99% bar chart
var_data = fred_all[["VaR 99%", "ES 99%"]].dropna().sort_values("ES 99%", ascending=False).head(15)
x = np.arange(len(var_data))
width = 0.4
axes[1, 0].barh(var_data.index, var_data["VaR 99%"], height=width, label="VaR 99%", color="#FF5722", alpha=0.8)
axes[1, 0].barh([i + width for i in range(len(var_data))], var_data["ES 99%"], height=width, label="ES 99%", color="#B71C1C", alpha=0.8)
axes[1, 0].set_xlabel("Loss (log returns)")
axes[1, 0].set_title("GPD Tail Risk: Top 15 by ES")
axes[1, 0].legend(fontsize=8)
axes[1, 0].tick_params(axis="y", labelsize=8)

# Panel 4: DEH gamma sorted bar chart
sorted_deh = fred_all["DEH gamma"].sort_values(ascending=False)
colors_deh = ["#E53935" if g > 0.2 else "#FF9800" if g > 0 else "#43A047" for g in sorted_deh]
axes[1, 1].barh(sorted_deh.index, sorted_deh.values, color=colors_deh, edgecolor="black", linewidth=0.3)
axes[1, 1].axvline(0, color="gray", linestyle="-", alpha=0.5)
axes[1, 1].set_xlabel("DEH gamma")
axes[1, 1].set_title("DEH Extreme Value Index (sorted)")
axes[1, 1].tick_params(axis="y", labelsize=8)

plt.suptitle("FRED Forex: 23 Currency Pairs — Key Metrics", fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

In [ ]:
# Summary statistics across all FRED pairs
fred_summary = fred_all.drop(columns=["N"]).agg(["mean", "median", "std", "min", "max"])

print("Summary Statistics: All FRED Forex Pairs")
print("=" * 100)
display(fred_summary.style.format("{:.4f}"))

# Comparison: FRED forex averages vs the 4 sample assets
print("\n\nComparison: Sample Assets vs FRED Forex Mean")
print("-" * 60)

sample_metrics = {}
for name in assets:
    r = returns[name]
    l = negative_returns(r)
    sample_metrics[name] = {
        "Hill alpha": hill_estimator(l),
        "DEH gamma": deh_estimator(l),
        "Hurst H": hurst_exponent(r),
    }

sample_metrics["FRED Mean"] = {
    "Hill alpha": fred_all["Hill alpha"].mean(),
    "DEH gamma": fred_all["DEH gamma"].mean(),
    "Hurst H": fred_all["Hurst H"].mean(),
}

comp_df = pd.DataFrame(sample_metrics).T
print(comp_df.to_string(float_format=lambda x: f"{x:.3f}"))

print("\n\nKey finding: FRED forex pairs have tail indices comparable to")
print("traditional assets (SPY, Gold) — confirming that fat tails are")
print("universal across financial markets, not just a crypto phenomenon.")

## 11. Summary

| Method | Measures | Interpretation | Best for |
|--------|----------|----------------|----------|
| **Hill estimator** | Tail index alpha | Lower alpha = fatter tails; alpha<2 infinite variance | Classic tail index estimation |
| **DEH estimator** | Tail index alpha | Same as Hill but more robust to second-order bias | When Hill is unreliable |
| **QQ estimator** | Tail index alpha | Log-log QQ regression-based alpha | Visual diagnostic + estimation |
| **Pickands estimator** | Tail index gamma | gamma>0 indicates fat tails; distribution-free | Non-parametric tail detection |
| **Max-stability Kappa** | Departure from Gaussian max-stability | Kappa below benchmark = fat tails | Quick Gaussian vs non-Gaussian test |
| **Taleb Kappa** | Same concept, reversed convention | Kappa above benchmark = fat tails | Taleb's formulation for practitioners |
| **Max-to-Sum Ratio** | Concentration of extremes | Higher ratio = single events dominate | Detecting infinite-variance regimes |
| **GPD fit** | Exceedance distribution (sigma, xi) | xi>0 = power-law tail | Threshold-based tail modelling |
| **GPD VaR/ES** | Value-at-Risk, Expected Shortfall | Tail risk quantification | Risk management |
| **GEV fit** | Block maxima distribution (mu, sigma, xi) | xi>0 = Frechet (fat tail) | Characterizing worst-case blocks |
| **Hurst exponent** | Long-range dependence H | H>0.5 persistent, H<0.5 mean-reverting | Detecting trending vs mean-reverting |
| **DFA exponent** | Detrended fluctuation scaling | Similar to Hurst, robust to trends | Persistence in non-stationary data |
| **Spectral exponent** | Power spectrum scaling beta | Higher = more persistence | Frequency-domain persistence analysis |

## 12. Crash Detection Accuracy: Precision, Recall & F1

The methods above measure **static properties** of returns (tail thickness, persistence).
But the real question is: **can they predict crashes?**

To answer this we run each method on **crash windows** (the 120 days before a known
drawdown peak) and **non-crash windows** (random 120-day stretches far from any crash).
This gives us a binary classification problem, and we measure it with three standard metrics.

### The confusion matrix

Every prediction falls into one of four boxes:

```
                        Actual crash?
                      YES           NO
                 ┌───────────┬───────────┐
  Method    YES  │    TP     │    FP     │
  fired?         │ (caught!) │ (false    │
                 │           │  alarm)   │
                 ├───────────┼───────────┤
            NO   │    FN     │    TN     │
                 │ (missed!) │ (correct  │
                 │           │  silence) │
                 └───────────┴───────────┘
```

- **TP** (True Positive): crash was coming, method fired — good
- **FP** (False Positive): no crash, method fired anyway — false alarm
- **FN** (False Negative): crash was coming, method stayed silent — missed it
- **TN** (True Negative): no crash, method stayed silent — good

### The three metrics

| Metric | Formula | Question it answers |
|--------|---------|---------------------|
| **Precision** | TP / (TP + FP) | "When the alarm goes off, how often is it real?" |
| **Recall** | TP / (TP + FN) | "Of all actual crashes, how many did we catch?" |
| **F1** | 2 * P * R / (P + R) | "Single score balancing both concerns" |

**The tradeoff**: you can trivially get 100% recall by always screaming "CRASH!" (but
precision goes to zero). You can get 100% precision by never firing (but recall goes to
zero). F1 is the harmonic mean — it punishes methods that are good at one but terrible
at the other.

In [ ]:
from analysis.accuracy_report import (
    find_drawdowns, sample_non_crash_windows,
    test_method_on_drawdown, test_method_on_non_crash,
    compute_metrics,
)

# ── Run the same evaluation as accuracy_report.py ──
tp_records = []
fp_records = []

all_crash_events = {}
for asset_name, threshold in [("btc", 0.15), ("spy", 0.08), ("gold", 0.08)]:
    df = assets[{"btc": "BTC", "spy": "SPY", "gold": "Gold"}[asset_name]]
    events = find_drawdowns(df, min_dd=threshold, min_apart=90)
    all_crash_events[asset_name] = events

    for ev in events:
        res, _ = test_method_on_drawdown(df, ev["peak_idx"])
        if res is None:
            continue
        for method, detected in res.items():
            if detected is not None:
                tp_records.append((method, detected))

for asset_name in ["btc", "spy", "gold"]:
    df = assets[{"btc": "BTC", "spy": "SPY", "gold": "Gold"}[asset_name]]
    events = all_crash_events[asset_name]
    for nc in sample_non_crash_windows(df, events, n_samples=50, seed=42):
        res, _ = test_method_on_non_crash(df, nc["center_idx"])
        if res is None:
            continue
        for method, fired in res.items():
            if fired is not None:
                fp_records.append((method, fired))

classical_methods = [
    "lppls", "lppls_confidence", "gsadf", "hurst", "dfa",
    "kappa", "taleb_kappa", "pickands", "deh", "qq",
    "gpd_var", "maxsum", "spectral", "hill",
]
metrics = compute_metrics(tp_records, fp_records, classical_methods)

n_crashes = sum(1 for m, d in tp_records if m == "lppls" and d is not None)
n_noncrash = sum(1 for m, d in fp_records if m == "lppls" and d is not None)
print(f"Evaluated on {n_crashes} crash windows and {n_noncrash} non-crash windows (BTC, SPY, Gold).\n")
print("All numbers are in-sample on historical data. This is not financial advice.")

In [ ]:
# ── Precision / Recall / F1 table ──
sorted_methods = sorted(classical_methods, key=lambda m: metrics.get(m, {}).get("f1", 0), reverse=True)

rows = []
for method in sorted_methods:
    m = metrics[method]
    if (m["tp"] + m["fn"]) == 0:
        continue
    rows.append({
        "Method": method,
        "TP": m["tp"], "FP": m["fp"], "FN": m["fn"], "TN": m["tn"],
        "Precision": m["precision"], "Recall": m["recall"], "F1": m["f1"],
    })

metrics_df = pd.DataFrame(rows)

# Format percentages for display
display_df = metrics_df.copy()
for col in ["Precision", "Recall", "F1"]:
    display_df[col] = display_df[col].map(lambda x: f"{x:.0%}")
print(display_df.to_string(index=False))

In [ ]:
# ── Visual: Precision vs Recall scatter ──
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: Precision vs Recall scatter
ax = axes[0]
for _, row in metrics_df.iterrows():
    ax.scatter(row["Recall"], row["Precision"], s=100, zorder=3)
    ax.annotate(row["Method"], (row["Recall"], row["Precision"]),
                textcoords="offset points", xytext=(6, 4), fontsize=8)

# F1 iso-lines
for f1_val in [0.2, 0.4, 0.6, 0.8]:
    r_range = np.linspace(0.01, 1, 200)
    p_range = f1_val * r_range / (2 * r_range - f1_val)
    mask = (p_range > 0) & (p_range <= 1)
    ax.plot(r_range[mask], p_range[mask], '--', color='grey', alpha=0.4, linewidth=0.8)
    # Label the iso-line
    idx = np.argmin(np.abs(r_range - 0.95))
    if mask[idx]:
        ax.text(0.96, p_range[idx], f"F1={f1_val:.1f}", fontsize=7, color='grey', va='center')

ax.set_xlabel("Recall (fraction of crashes caught)")
ax.set_ylabel("Precision (fraction of alerts that are correct)")
ax.set_title("Precision vs Recall (dashed lines = F1 iso-curves)")
ax.set_xlim(0, 1.05)
ax.set_ylim(0, 0.7)

# Right: F1 bar chart
ax2 = axes[1]
colors = ['#2196F3' if f1 > 0.4 else '#FF9800' if f1 > 0.25 else '#9E9E9E'
          for f1 in metrics_df["F1"]]
ax2.barh(metrics_df["Method"], metrics_df["F1"], color=colors)
ax2.set_xlabel("F1 Score")
ax2.set_title("F1 Score by Method (sorted best to worst)")
ax2.invert_yaxis()
for i, (f1, prec, rec) in enumerate(zip(metrics_df["F1"], metrics_df["Precision"], metrics_df["Recall"])):
    ax2.text(f1 + 0.01, i, f"P={prec:.0%} R={rec:.0%}", va='center', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# ── Confusion matrix heatmap for LPPLS (the best method) ──
lppls_m = metrics["lppls"]

cm = np.array([
    [lppls_m["tp"], lppls_m["fp"]],
    [lppls_m["fn"], lppls_m["tn"]],
])

fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(cm, cmap="YlOrRd", aspect="auto")

labels = [
    [f"TP\n{cm[0,0]}\n(caught!)", f"FP\n{cm[0,1]}\n(false alarm)"],
    [f"FN\n{cm[1,0]}\n(missed!)", f"TN\n{cm[1,1]}\n(correct silence)"],
]

for i in range(2):
    for j in range(2):
        color = "white" if cm[i, j] > cm.max() * 0.6 else "black"
        ax.text(j, i, labels[i][j], ha="center", va="center",
                fontsize=12, fontweight="bold", color=color)

ax.set_xticks([0, 1])
ax.set_xticklabels(["Actual: CRASH", "Actual: NO CRASH"])
ax.set_yticks([0, 1])
ax.set_yticklabels(["Predicted:\nCRASH", "Predicted:\nNO CRASH"])
ax.set_title(f"LPPLS Confusion Matrix\nP={lppls_m['precision']:.0%}  R={lppls_m['recall']:.0%}  F1={lppls_m['f1']:.0%}")
plt.tight_layout()
plt.show()

print("Reading the confusion matrix:")
print(f"  - LPPLS correctly catches {lppls_m['tp']} of {lppls_m['tp']+lppls_m['fn']} crashes (recall = {lppls_m['recall']:.0%})")
print(f"  - But it also fires {lppls_m['fp']} times when there's no crash (precision = {lppls_m['precision']:.0%})")
print(f"  - For crash detection, high recall is arguably more important than high precision:")
print(f"    missing a crash (FN) is worse than a false alarm (FP).")

### Why LPPLS dominates

LPPLS (Log-Periodic Power Law Singularity) is the only method here that models the
**mechanism** of a crash: super-exponential price growth with accelerating oscillations
converging to a critical time tc. The other methods detect **symptoms** (fat tails,
persistence, regime shifts) that are necessary but not sufficient for an imminent crash.

This is why LPPLS achieves 90% recall with the best F1 — it's the only method that
directly models the bubble-to-crash transition, rather than just measuring statistical
properties of returns.

### Why no method has high precision

All methods have precision below 55%. This is fundamental, not a bug:
- Markets can exhibit bubble-like patterns (super-exponential growth, fat tails,
  persistence) without crashing — the bubble can deflate slowly instead
- The same statistical signatures appear in bull markets that continue for years
- Crash timing is inherently uncertain — a method that fires "too early" registers
  as a false positive even though the underlying signal was correct

This is why the aggregator approach matters: combining independent signal categories
(bubble dynamics, tail risk, persistence, structural) reduces false positives when
multiple categories agree.

*All numbers are in-sample on historical data. This is not financial advice.*

## 13. Beyond Valuations: Applicability to Revenue & Profit

All the methods above were tested on **market prices** (BTC, SPY, Gold, FX). But do they
work on **fundamental data** like revenue or profit growth?

The short answer: **some do, some don't.** It depends on whether the method measures a
general statistical property of time series or a market-specific mechanism.

### Which methods transfer

| Method | Transfers? | Why |
|--------|-----------|-----|
| **Hill, DEH, QQ, Pickands** (tail index) | Yes | Tail thickness is a property of any distribution. Revenue growth rates have fat tails too — Gabaix (2011) showed that idiosyncratic firm-level shocks (granular origins) drive aggregate fluctuations precisely because firm-size distributions are fat-tailed. |
| **Kappa, Taleb kappa** | Yes | Measures departure from Gaussian max-stability. Works on any data where you suspect non-Gaussian extremes. |
| **Max-to-Sum ratio** | Yes | Tests whether single observations dominate the total. A single quarter where revenue drops 50% dominating the sum = same math as a single market crash day. |
| **GPD / GEV** | Yes | EVT is distribution-agnostic. Fitting GPD to the worst quarterly revenue declines gives valid VaR/ES estimates for fundamental risk. |
| **Hurst, DFA, Spectral** (persistence) | Yes | Revenue series are often *strongly* persistent (H > 0.5) due to contracts, customer stickiness, and operating leverage. A shift from persistent to anti-persistent (H dropping below 0.5) could signal fundamental deterioration. |
| **GSADF** (explosive roots) | Partially | Detects unsustainable exponential growth. Applied to quarterly revenue, it could flag "revenue bubbles" in hypergrowth companies — growth rates that imply the company would need to capture 100% of its TAM. |
| **LPPLS** | No | Models speculative bubble dynamics (herding, positive feedback, log-periodic oscillations). Revenue doesn't exhibit these patterns — it's driven by real economic activity, not reflexive speculation. |
| **LPPLS confidence** | No | Same limitation as LPPLS. |

### The key distinction

**Market prices** reflect collective speculative behavior — they have reflexivity
(Soros), herding, and positive feedback loops. This is what LPPLS models.

**Revenue/profit** reflects real economic activity — customer demand, operational
execution, competitive dynamics. The feedback loops are different: a company's revenue
doesn't go up because investors *believe* it will go up (unlike stock prices).

This means:
- **Tail estimators** work on both because fat tails are universal (Mandelbrot, Taleb)
- **Persistence methods** work on both because autocorrelation is a general property
- **Bubble detectors** (LPPLS, GSADF) are price-specific — they need speculative dynamics

### What you'd measure in practice

For a company's **quarterly revenue** time series:
1. **Log-changes**: `r_t = log(revenue_t / revenue_{t-1})` — the growth rate per quarter
2. **Tail index**: Hill/DEH alpha on the growth rates — are revenue shocks fat-tailed?
3. **Persistence**: Hurst/DFA on growth rates — is growth momentum persistent or mean-reverting?
4. **EVT**: GPD on the worst declines — what's the 95th percentile revenue drop?
5. **GSADF**: on the revenue level — is growth explosive/unsustainable?

The challenge: quarterly data gives you ~80 observations over 20 years (vs ~5,000 daily
prices). Tail estimators need at least ~100 data points to be reliable, so you may need
monthly revenue or longer history.

*This is not financial advice. For research and educational purposes only.*